In [3]:
import requests as req
from bs4 import BeautifulSoup
import json
import math
import pandas as pd
import re
import numpy as np
import os
import timeit

from pprint import pprint as pp

In [4]:
def get_request_ug(url):
    '''Makes a get request to the specified Ultimate Guitar URL and returns relevant JSON data'''
    
    # Send request
    res = req.get(url)
    
    # Extract tab data from response
    # Note: I didn't bother with error handling in this instance, but you could check response code == 200, for example
    soup = BeautifulSoup(res.content) # format response content
    
    # Use BeautifulSoup to find relevant element, then cast to json and drill down the Ultimate-Guitar-Specific heirarchy
    page_data = json.loads(soup.find("div", {'class': 'js-store'})['data-content'])['store']['page']['data']
    
    return page_data

In [5]:
def scrape_page(page):
    # query params
    order = 'hitstotal_desc'  # order by total hits, descending order. i.e. most popular -> least popular
    tab_type = 'Chords'  # filter tabs that provide chord progression information (case sensitive)
    path = 'https://www.ultimate-guitar.com/explore'  # base endpoint path
    
    # generate url for request
    query_params = 'order=' + order + '&page=' + str(page)  + '&type[]=' + tab_type
    url = path + '?' + query_params
    
    # make request
    page_data = get_request_ug(url)['data']
    
    # create hits dataframe (stored separately to tab data)
    hits = page_data['hits']  # hits are stored with unique id which is also found in tab data
    hits = [[int(r['id']), int(r['hits'])] for r in hits]  # prep list of lists to be read into pandas dataframe, cast id and hits to integer to align with tabs dataset
    hits = pd.DataFrame(hits, columns = ['id', 'hits']).set_index('id')  # set index to shared unique id

    # create tabs dataframe (one page of data)
    tabs = page_data['tabs']  # extract all tabs and their attributes
    df = pd.DataFrame(tabs, columns=tabs[0].keys()).set_index('id')  # set index to shared unique id

    # join hits and tab data together for one page
    df = df.join(hits)  # adds hits dataframe to tabs dataframe - left join takes place by default on (shared) index
    return df

In [6]:
def get_tab_metadata(num_tabs):  
    #calc number of pages to scrape
    num_pages = math.ceil(num_tabs / 50)  # 50 tabs per page
    
    # initiate master dataframe
    df = pd.DataFrame()

    for p in range(num_pages):
        page_tabs = scrape_page(p)
        df = df.append(page_tabs)
    return df

In [7]:
def get_chords(song_data):
    '''
    get_chords takes song data, and
    - extracts each of the following sections: Intro, Verse 1-9, Chorus, Outro, Bridge
    - parses each section for chords
    - returns 
        if tab in correct format: a dataframe with two columns [section, chords]
        otherwise: empty dataframe
    output columns
    - section: contains the name of each section eg. Chorus (may be duplicated)
    - chords: contians a list of chords eg. ['Eb', 'Gm6', 'Fsus4', 'Bb']
    '''
    
    # regex patterns
    extract_verse_name = r'\[[VCIOB][ehnur][roti][srd][euog][se ]?(\d?)\]'  # extract name of section. eg. Chorus
    extract_verse_with_name = extract_verse_name + r'\\r\\n(.*?)\\r\\n\\r\\n'  # extract each verse name and tab content
    extract_chords = r'\[ch\](.*?)\[\/ch\]'  # extract chords from within chord and lyric data
    
    song_data = song_data.replace('\r', '\\r').replace('\n', '\\n');  # to work with regex parser
    sections = re.finditer(extract_verse_with_name, song_data)  # has form [ '[section_name]', 'lyrics_and_chords' ]

    song = []  # intialise to hold data for each section
    for s in sections:
        section_string = song_data[s.start(): s.end()]  # extract a verse (with its name) from song_data
        section = re.search(extract_verse_name, section_string)  # get verse name from string
        
        section_name = section.group(0)[1:-1]  # remove []  surrounding section_name
        chords = re.findall(extract_chords, section_string)  # extract chords from lyrics
        section = {'section_name': section_name, 'chords': chords}  # save section name with its chords
        song.append(section)  # append to get a view for the whole song

    return pd.DataFrame(song)  # turn song into dataframe


In [8]:
# add HTML explaining chord nomenclature

In [20]:
def to_triad(chord):
    '''
    convert a chord into a three-note chord (triad)
    eg. A7sus4 -> A
    eg. E#dim7 -> E#dim
    '''
    is_dim = False  # indicates a diminished chord
    
    # systematically take everything above chord modifiers, from right to left
    # if the modifiers don't exist, then the chord is returned unchanged
    if '/' in chord:  # eg. A#m7/F -> A#m7
        chord = chord.split('/')[0]
    if 'sus' in chord: # eg. Csus2 -> C
        chord = chord.split('sus')[0]
    if 'add' in chord: # eg. Fmadd9 -> Fm
        chord = chord.split('add')[0]
    if 'maj' in chord:  # eg. Bbmaj7 -> Bb
        chord = chord.split('maj')[0]
    if 'dim' in chord:  # eg. A7dim -> A7
        chord = chord.split('dim')[0]
        is_dim = True  # flag for later, diminished chords are basic triads
        
    # at this point the chord is a simplified triad, except for the last position sometimes indicating \
    # an extension on a triad chord (not a pure triad)
    if chord[-1].isnumeric():
        chord = chord[:-1]
    
    # append 'dim' modifier if applicable
    if is_dim:
        chord = chord + 'dim'
        
    return chord
   

def get_section_score(section, key_chords):
    '''
    returns number of matches between a sequence of chords and a set of chords
    '''
    return sum([1 for chord in section if chord in set(key_chords)])


def get_key(section):
    '''
    get_key takes a list of triad chords
    returns 
        keys: an array of possible keys
    multiple keys are possible where a section has limited chords \
    (eg. only 2 chords that are shared among multiple keys)
    '''

    scores = []
    for key in key_chord_mapping.columns:
        key_chords = key_chord_mapping[key]  # get chords corresponding to key
        score = get_section_score(section, key_chords)  # get match score for section chords against this key
        
        item = {'key': key, 'score': score}
        scores.append(item)
    key = max(scores, key=lambda x: x['score'])['key']
    
    return key


def guitar_to_piano_chords(chords, capo):
    '''
    takes a list of guitar chords and adjusts them to piano based on the fret that the capo is on (1 semitone per fret)
    returns
        pitched_up_chords: a list of chords raised by n=capo semitones
    '''
    
    chords_pitch_adj = []
    for chord in chords:
        for semitone in range(capo):
            chord = adjust_pitch_by_semitone(chord=chord, pitch_up=True)
        chords_pitch_adj.append(chord)
   
    return chords_pitch_adj


def is_minor(chord):
    if chord[-1] == 'm':  # chord is minor
        return True
    else:
        return False

    
def adjust_pitch_by_semitone(chord, pitch_up):
    '''
    takes a chord and pitches_up based on value or pitch_up (boolean)
    returns
        pitch-adjusted chord
    '''
    
    minor = is_minor(chord)
    
    if minor:
        chord = chord[:-1]  # chop off m
        
    if pitch_up:
        switcher = {
            'A': 'A#',
            'A#': 'B',
            'Bb': 'B',
            'B': 'C',
            'C': 'C#',
            'C#': 'D',
            'Db': 'D',
            'D': 'D#',
            'D#': 'E',
            'Eb': 'E',
            'E': 'F',
            'F': 'F#',
            'F#': 'G',
            'Gb': 'G',
            'G': 'G#',
            'G#': 'A',
            'Ab': 'A',
        }
        
    elif not pitch_up:
        switcher = {
            'A#': 'A',
            'B': 'A#',
            'B': 'Bb',
            'C': 'B',
            'C#': 'C',
            'D': 'C#',
            'D': 'Db',
            'D#': 'D',
            'E': 'D#',
            'E': 'Eb',
            'F': 'E',
            'F#': 'F',
            'G': 'F#',
            'G': 'Gb',
            'G#': 'G',
            'A': 'G#',
            'A': 'Ab',
        }
        
    
    else:  # pitch_up not boolean, so return original chord
        return chord
    
    # get adjusted chord value
    chord = switcher.get(chord, None)

    if minor:
        chord = chord + 'm'  # add m back
    
    return chord


def chords_to_degree(chords, key, map_to_closest=False):
    '''
    chords_to_degree(chords, key)
    - maps chords in key provided scale degree (index of key_chord_mapping)
    '''
    chords_in_key = key_chord_mapping[key]
    chord_degrees = []
    print(f'1 {chords}')
    print(f'2 {chords_in_key.to_list()}')
    for chord in chords:
        if chord in chords_in_key.to_list():
            degree = key_chord_mapping[chords_in_key==chord].index.to_numpy()[0]
        elif map_to_closest:
            degree = closest_chord_in_key(chord, key)
        else:
            degree =  None
        chord_degrees.append(degree)
    print(f'3 {chord_degrees}')
    print()
    
    return chord_degrees


def flat_to_sharp(chord):
    flat_to_sharp = {
        'Ab': 'G#',
        'Bb': 'A#',
        'Cb': 'Bb',
        'Db': 'B',
        'Eb': 'C',
        'Fb': 'C#',
        'Gb': 'Db'
    }
    
    return flat_to_sharp.get(chord, chord)


def sharp_to_flat(chord):
    sharp_to_flat = {
        'A#': 'Bb',
        'B#': 'Cb',
        'C#': 'Db',
        'D#': 'Eb',
        'E#': 'Fb',
        'F#': 'Gb',
        'G#': 'Ab' 
    }
    
    return sharp_to_flat.get(chord, chord)


def switch_notation(chord):
    """
    take a chord and switch to its equivalent notation 
    eg. A# -> Bb, Cb -> B, A#m -> G#m -> Abm
    if chord is not flat or sharp, pass through
    """
    
    minor = is_minor(chord)
    
    if minor:
        chord = chord[:-1]  # chop off m
        
    notation_switcher = {
        'A#': 'Bb',
        'D#': 'Eb',
        'G#': 'Ab',
        'C#': 'Db',
        'F#': 'Gb',
        'B': 'Cb',
        'Cb': 'B',
        'Gb': 'F#',
        'Db': 'C#',
        'Ab': 'G#',
        'Eb': 'D#',
        'Bb': 'A#'
    }
    
    chord = notation_switcher.get(chord, chord)

    if minor:
        chord = chord + 'm'
        
    return chord
    
    
def is_flat(chord):
    if len(chord) > 1 and chord[1] == 'b':
        return True
   
    return False


def is_sharp(chord):
    if len(chord) > 1 and chord[1] == '#':
        return True
   
    return False


def closest_chord_in_key(chord, key): ###### TODO
    
    chords_in_key = key_chord_mapping[key]
    
    if chord in chords_in_key:
        return chord
    
    
    
    return None
    # A-G  #bm  m

In [22]:
# User input
num_tabs = 50  # number of tabs wanted for dataset (will be rounded up to nearest 50 as tabs are scraped per page)

# End user input

DATA_DIR = os.path.join(os.getcwd(), 'data')
KEY_CHORD_MAPPING_DIR = os.path.join(DATA_DIR, 'lookups')
key_chord_mapping = pd.read_csv(os.path.join(KEY_CHORD_MAPPING_DIR, 'musical_key-triad_chord_mapping.csv'), index_col='Degree')

md = get_tab_metadata(num_tabs)  # scrape ultimate guitar 'explore' page for metadata (md)
md.drop(labels=['recording', 'part', 'tab_access_type', 'version_description', 'tp_version'], axis=1, inplace=True)

chords = []
# loop through tabs, fetch tab data and parse into chords
for index, tab_url in md['tab_url'].iteritems():  # get index to use as foreign key
    
    # send get request and extract tab info
    tab_data = get_request_ug(tab_url)['tab_view']  # get tab data
    tab_content = tab_data['wiki_tab']['content']  # extract tab content
    
    # extract capo fret number to pitch up guitar chords to get the true chord
    try: 
        # eg. capo on 2nd fret, a chord that is denoted as Em is actually F#m
        capo = tab_data['meta']['capo']
    except KeyError: 
        # capo key not present in json response
        capo = 0
        
    # parse tab content to extract chord
    song = get_chords(tab_content)
    
    # create additional columns, or move to next URL if song is empty
    if song.empty:
        continue
    else:  # list multiplication to create new column of correct length
        song['id'] = pd.Series([index] * len(song), dtype='int64')  # to be used to reference metadata (foreign key)
        song['capo'] = pd.Series([capo] * len(song), dtype='int64')  # save capo with chords
        song['key'] = pd.Series([None] * len(song), dtype='str')
        song['chords_simplified'] = pd.Series([None] * len(song), dtype='str')
        song['chords_simplified_pitch_corrected'] = pd.Series([None] * len(song), dtype='str')
        song['chords_numeric'] = pd.Series([None] * len(song), dtype='str')
    
    
    # get alternate chord representations
    song_chords = []
    for ids, section in song['chords'].iteritems():
        simple_section = []
        simple_pitch_corrected_section = []
        for chord in section:
            simple_chord = to_triad(chord)
            simple_section.append(simple_chord)
        
        # pitch correct guitar chords
        simple_pitch_corrected_section = guitar_to_piano_chords(simple_section, capo)
        
        # add alternate chords to dataframe
        song.at[ids, 'chords_simplified'] = simple_section
        song.at[ids, 'chords_simplified_pitch_corrected'] = simple_pitch_corrected_section
        
        song_chords.extend(simple_pitch_corrected_section)

    # get key of song - assumed 1 chord for entire song
    key = get_key(song_chords)
    
    # check if key (and chords) can be simplified
    key_changed = False
    if key in ['A#', 'D#', 'G#', 'C#', 'F#']:  # these keys can be represented more simply
        key = sharp_to_flat(key)
        key_changed = True
    elif key in ['Cb']: # check separately to sharpened keys to avoid needing to check if key is flat or sharp 
        key = flat_to_sharp(key)
        key_changed = True
    if key_changed:  # update chords
        for ids, section in song['chords_simplified_pitch_corrected'].iteritems():
            sec = []
            for chord in section:
                sec.append(switch_notation(chord))
        song.at[ids, 'chords_simplified_pitch_corrected'] = sec     
    
    # add song's key to each of its sections in the dataframe
    song['key'] = [key] * len(song)
    
    # loop through sections and make numeric
    for ids, section in song['chords_simplified_pitch_corrected'].iteritems():
        song.at[ids, 'chords_numeric'] = chords_to_degree(section, key, map_to_closest=True)
        
    # create chords: a list of song dataframes
    chords.append(pd.DataFrame(song))  

1 ['C#', 'A#m', 'C#', 'A#m']
2 ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cdim']
3 [None, None, None, None]

1 ['C#', 'A#m', 'C#', 'A#m', 'F#', 'G#', 'C#', 'G#', 'C#', 'F#', 'G#', 'A#m', 'F#', 'G#', 'F', 'A#m']
2 ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cdim']
3 [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]

1 ['F#', 'A#m', 'F#', 'C#', 'G#', 'C#', 'A#m', 'C#', 'A#m']
2 ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cdim']
3 [None, None, None, None, None, None, None, None, None]

1 ['C#', 'A#m', 'C#', 'A#m', 'F#', 'G#', 'C#', 'G#', 'C#', 'F#', 'G#', 'A#m', 'F#', 'G#', 'F', 'A#m']
2 ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cdim']
3 [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]

1 ['F#', 'A#m', 'F#', 'C#', 'G#', 'C#', 'A#m', 'C#', 'A#m']
2 ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cdim']
3 [None, None, None, None, None, None, None, None, None]

1 ['C#', 'A#m', 'C#', 'A#m', 'F#', 'G#', 'C#', 'G#', 'C#',

1 ['A#m', 'G#', 'C#', 'A#m', 'G#', 'C#']
2 ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cdim']
3 [None, None, None, None, None, None]

1 ['A#m', 'G#', 'C#', 'A#m', 'G#', 'C#', 'A#m', 'G#', 'C#', 'A#m', 'G#', 'C#']
2 ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cdim']
3 [None, None, None, None, None, None, None, None, None, None, None, None]

1 ['A#m', 'G#', 'C#', 'A#m', 'G#', 'C#', 'A#m', 'G#', 'C#', 'A#m', 'G#', 'C#']
2 ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cdim']
3 [None, None, None, None, None, None, None, None, None, None, None, None]

1 ['A#m', 'G#', 'C#', 'A#m', 'G#', 'C#', 'A#m', 'G#', 'C#', 'A#m', 'G#', 'C#']
2 ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cdim']
3 [None, None, None, None, None, None, None, None, None, None, None, None]

1 ['A#m', 'G#', 'C#', 'A#m', 'G#', 'C#', 'A#m', 'G#', 'C#', 'A#m', 'G#', 'C#']
2 ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cdim']
3 [None, None, None, None, None, None, None, None, None, None, None, None]

1 ['A#m', 'G#', 'C#', 'F#', 'A#m', 'G#', 'C#', 'F#']
2

1 ['D', 'A', 'Bm', 'G']
2 ['D', 'Em', 'F#m', 'G', 'A', 'Bm', 'C#dim']
3 [1, 5, 6, 4]

1 ['D', 'G', 'Bm', 'G', 'D', 'G', 'Bm', 'A']
2 ['D', 'Em', 'F#m', 'G', 'A', 'Bm', 'C#dim']
3 [1, 4, 6, 4, 1, 4, 6, 5]

1 ['D', 'A', 'Bm', 'G', 'A', 'D']
2 ['D', 'Em', 'F#m', 'G', 'A', 'Bm', 'C#dim']
3 [1, 5, 6, 4, 5, 1]

1 ['D', 'G', 'Bm', 'A']
2 ['D', 'Em', 'F#m', 'G', 'A', 'Bm', 'C#dim']
3 [1, 4, 6, 5]

1 ['D', 'A', 'Bm', 'G', 'A', 'D', 'A', 'Bm', 'G', 'A']
2 ['D', 'Em', 'F#m', 'G', 'A', 'Bm', 'C#dim']
3 [1, 5, 6, 4, 5, 1, 5, 6, 4, 5]

1 ['Bm', 'G', 'D', 'A', 'Bm', 'G', 'D', 'A']
2 ['D', 'Em', 'F#m', 'G', 'A', 'Bm', 'C#dim']
3 [6, 4, 1, 5, 6, 4, 1, 5]

1 ['D', 'A', 'Bm', 'G', 'A', 'E', 'B', 'C#m', 'A', 'B', 'E']
2 ['D', 'Em', 'F#m', 'G', 'A', 'Bm', 'C#dim']
3 [1, 5, 6, 4, 5, None, None, None, 5, None, None]

1 ['E', 'B', 'C#m', 'A']
2 ['E', 'F#m', 'G#m', 'A', 'B', 'C#m', 'D#dim']
3 [1, 5, 6, 4]

1 ['E', 'B', 'C#m', 'A', 'E', 'B', 'C#m', 'A', 'B', 'E', 'B', 'C#m', 'A', 'E', 'B', 'C#m', 'A', 'B']
2 ['

1 ['C#', 'D#', 'G#', 'Fm']
2 ['Ab', 'Bbm', 'Cm', 'Db', 'Eb', 'Fm', 'Gdim']
3 [None, None, None, 6]

1 ['Fm', 'C#', 'D#', 'G#', 'Fm', 'C#', 'D#', 'G#', 'Fm']
2 ['Ab', 'Bbm', 'Cm', 'Db', 'Eb', 'Fm', 'Gdim']
3 [6, None, None, None, 6, None, None, None, 6]

1 ['Fm', 'C#', 'D#', 'G#', 'Fm', 'C#', 'D#', 'G#', 'Fm', 'Fm', 'C#', 'D#', 'G#', 'Fm', 'C#', 'D#', 'G#', 'Fm']
2 ['Ab', 'Bbm', 'Cm', 'Db', 'Eb', 'Fm', 'Gdim']
3 [6, None, None, None, 6, None, None, None, 6, 6, None, None, None, 6, None, None, None, 6]

1 ['C#', 'D#', 'G#', 'Fm', 'C#', 'D#', 'G#', 'Fm', 'C#', 'D#', 'G#', 'Fm', 'C#', 'D#', 'Cm', 'Fm']
2 ['Ab', 'Bbm', 'Cm', 'Db', 'Eb', 'Fm', 'Gdim']
3 [None, None, None, 6, None, None, None, 6, None, None, None, 6, None, None, 3, 6]

1 ['Fm', 'C#', 'D#', 'G#', 'Fm', 'C#', 'D#', 'G#', 'Fm', 'Fm', 'C#', 'D#', 'G#', 'Fm', 'C#', 'D#', 'G#', 'Fm']
2 ['Ab', 'Bbm', 'Cm', 'Db', 'Eb', 'Fm', 'Gdim']
3 [6, None, None, None, 6, None, None, None, 6, 6, None, None, None, 6, None, None, None, 6]

1 ['C#',

1 ['Am', 'F', 'C', 'Am', 'F', 'C']
2 ['C', 'Dm', 'Em', 'F', 'G', 'Am', 'Bdim']
3 [6, 4, 1, 6, 4, 1]

1 ['Am', 'F', 'C', 'Am', 'F', 'C', 'Am', 'F', 'C', 'Am', 'G', 'C']
2 ['C', 'Dm', 'Em', 'F', 'G', 'Am', 'Bdim']
3 [6, 4, 1, 6, 4, 1, 6, 4, 1, 6, 5, 1]

1 ['Am', 'F', 'C', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G#dim', 'Am', 'F', 'C']
2 ['C', 'Dm', 'Em', 'F', 'G', 'Am', 'Bdim']
3 [6, 4, 1, 6, 4, 1, 5, 6, 4, 1, None, 6, 4, 1]

1 ['Am', 'F', 'C', 'Am', 'F', 'C', 'Am', 'F', 'C', 'Am', 'G', 'C']
2 ['C', 'Dm', 'Em', 'F', 'G', 'Am', 'Bdim']
3 [6, 4, 1, 6, 4, 1, 6, 4, 1, 6, 5, 1]

1 ['Am', 'F', 'C', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G#dim', 'Am', 'F', 'C']
2 ['C', 'Dm', 'Em', 'F', 'G', 'Am', 'Bdim']
3 [6, 4, 1, 6, 4, 1, 5, 6, 4, 1, None, 6, 4, 1]

1 ['Am', 'F', 'C', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G#dim', 'Am', 'F', 'C']
2 ['C', 'Dm', 'Em', 'F', 'G', 'Am', 'Bdim']
3 [6, 4, 1, 6, 4, 1, 5, 6, 4, 1, None, 6, 4, 1]

1 ['Am', 'F', 'C', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G#dim', 'Am', 'F',

1 ['A', 'C#m', 'F#m', 'D']
2 ['A', 'Bm', 'C#m', 'D', 'E', 'F#m', 'G#dim']
3 [1, 3, 6, 4]

1 ['A', 'C#m', 'F#m', 'D', 'A', 'C#m', 'F#m', 'D', 'A', 'C#m', 'F#m', 'D']
2 ['A', 'Bm', 'C#m', 'D', 'E', 'F#m', 'G#dim']
3 [1, 3, 6, 4, 1, 3, 6, 4, 1, 3, 6, 4]

1 ['A', 'E', 'F#m', 'D', 'A', 'E', 'F#m', 'D', 'A', 'E', 'F#m', 'D', 'A', 'E', 'F#m', 'D', 'A', 'E', 'F#m', 'D']
2 ['A', 'Bm', 'C#m', 'D', 'E', 'F#m', 'G#dim']
3 [1, 5, 6, 4, 1, 5, 6, 4, 1, 5, 6, 4, 1, 5, 6, 4, 1, 5, 6, 4]

1 ['A', 'C#m', 'F#m', 'D', 'A', 'C#m', 'F#m', 'D']
2 ['A', 'Bm', 'C#m', 'D', 'E', 'F#m', 'G#dim']
3 [1, 3, 6, 4, 1, 3, 6, 4]

1 ['A', 'E', 'F#m', 'D', 'A', 'E', 'F#m', 'D', 'A', 'E', 'F#m', 'D', 'A', 'E', 'F#m', 'D']
2 ['A', 'Bm', 'C#m', 'D', 'E', 'F#m', 'G#dim']
3 [1, 5, 6, 4, 1, 5, 6, 4, 1, 5, 6, 4, 1, 5, 6, 4]

1 ['E', 'F#m', 'D', 'Bm', 'A', 'D', 'E']
2 ['A', 'Bm', 'C#m', 'D', 'E', 'F#m', 'G#dim']
3 [5, 6, 4, 2, 1, 4, 5]

1 ['A', 'E', 'F#m', 'D', 'A', 'E', 'F#m', 'D', 'A', 'E', 'F#m', 'D', 'A', 'E', 'F#m', 'D']
2 ['

In [23]:
chords = pd.concat(chords, axis=0, ignore_index=True)

In [12]:
chords

,section_name,chords,id,capo,key,chords_simplified,chords_simplified_pitch_corrected,chords_numeric
0,Intro,"[C, Am, C, Am]",198052,1,C#,"[C, Am, C, Am]","[C#, A#m, C#, A#m]","[None, None, None, None]"
1,Verse 1,"[C, Am, C, Am, F, G, C, G, C, F, G, Am, F, G, ...",198052,1,C#,"[C, Am, C, Am, F, G, C, G, C, F, G, Am, F, G, ...","[C#, A#m, C#, A#m, F#, G#, C#, G#, C#, F#, G#,...","[None, None, None, None, None, None, None, Non..."
2,Chorus,"[F, Am, F, C, G, C, Am, C, Am]",198052,1,C#,"[F, Am, F, C, G, C, Am, C, Am]","[F#, A#m, F#, C#, G#, C#, A#m, C#, A#m]","[None, None, None, None, None, None, None, Non..."
3,Verse 2,"[C, Am, C, Am, F, G, C, G, C, F, G, Am, F, G, ...",198052,1,C#,"[C, Am, C, Am, F, G, C, G, C, F, G, Am, F, G, ...","[C#, A#m, C#, A#m, F#, G#, C#, G#, C#, F#, G#,...","[None, None, None, None, None, None, None, Non..."
4,Chorus,"[F, Am, F, C, G, C, Am, C, Am]",198052,1,C#,"[F, Am, F, C, G, C, Am, C, Am]","[F#, A#m, F#, C#, G#, C#, A#m, C#, A#m]","[None, None, None, None, None, None, None, Non..."
...,...,...,...,...,...,...,...,...
308,Verse 1,"[Em, G, D, A7sus4, Em, G, D, A7sus4]",27596,2,A,"[Em, G, D, A, Em, G, D, A]","[F#m, A, E, B, F#m, A, E, B]","[6, 1, 5, None, 6, 1, 5, None]"
309,Verse 2,"[Em, G, D, A7sus4, Em, G, D, A7sus4, Em, G, D,...",27596,2,A,"[Em, G, D, A, Em, G, D, A, Em, G, D, A, Em, G,...","[F#m, A, E, B, F#m, A, E, B, F#m, A, E, B, F#m...","[6, 1, 5, None, 6, 1, 5, None, 6, 1, 5, None, ..."
310,Chorus,"[C, Em, G, Em, C, Em, G, Em, C, Em, G, Em, C, ...",27596,2,A,"[C, Em, G, Em, C, Em, G, Em, C, Em, G, Em, C, ...","[D, F#m, A, F#m, D, F#m, A, F#m, D, F#m, A, F#...","[4, 6, 1, 6, 4, 6, 1, 6, 4, 6, 1, 6, 4, 6, 1, 6]"
311,Verse 3,"[Em, G, D, A7sus4, Em, G, D, A7sus4, Em, G, D,...",27596,2,A,"[Em, G, D, A, Em, G, D, A, Em, G, D, A, Em, G,...","[F#m, A, E, B, F#m, A, E, B, F#m, A, E, B, F#m...","[6, 1, 5, None, 6, 1, 5, None, 6, 1, 5, None, ..."


In [13]:
md[md['tab_url'] == tab_url]

,song_id,song_name,artist_id,artist_name,type,version,votes,rating,date,status,preset_id,tonality_name,verified,artist_url,tab_url,type_name,hits
id,,,,,,,,,,,,,,,,,
27596,19556,Wonderwall,734,Oasis,Chords,2,5205,4.77542,1018656000,approved,2332,F#m,0,https://www.ultimate-guitar.com/artist/oasis_734,https://tabs.ultimate-guitar.com/tab/oasis/won...,Chords,7935922


In [14]:
md.head()

,song_id,song_name,artist_id,artist_name,type,version,votes,rating,date,status,preset_id,tonality_name,verified,artist_url,tab_url,type_name,hits
id,,,,,,,,,,,,,,,,,
198052,135952,Hallelujah,9898,Jeff Buckley,Chords,2,44749,4.87497,1121385600,approved,125,Db,0,https://www.ultimate-guitar.com/artist/jeff_bu...,https://tabs.ultimate-guitar.com/tab/jeff-buck...,Chords,33597179
1956589,1710759,Perfect,30232,Ed Sheeran,Chords,1,36518,4.86027,1488453515,approved,18913,Ab,0,https://www.ultimate-guitar.com/artist/ed_shee...,https://tabs.ultimate-guitar.com/tab/ed-sheera...,Chords,29508463
1248578,346275,All Of Me,11714,John Legend,Chords,1,22016,4.83865,1369872001,approved,14737,Em,0,https://www.ultimate-guitar.com/artist/john_le...,https://tabs.ultimate-guitar.com/tab/john-lege...,Chords,26714219
1137467,298037,Let Her Go,21762,Passenger,Chords,1,19655,4.85244,1331596801,approved,2333,Em,0,https://www.ultimate-guitar.com/artist/passeng...,https://tabs.ultimate-guitar.com/tab/passenger...,Chords,26584704
1086983,152656,Cant Help Falling In Love,11125,Elvis Presley,Chords,1,23728,4.86595,1314230401,approved,2368,C,0,https://www.ultimate-guitar.com/artist/elvis_p...,https://tabs.ultimate-guitar.com/tab/elvis-pre...,Chords,23493010


In [15]:
song.head()

,section_name,chords,id,capo,key,chords_simplified,chords_simplified_pitch_corrected,chords_numeric
0,Intro,"[Em, G, D, A7sus4, Em, G, D, A7sus4, Em, G, D,...",27596,2,A,"[Em, G, D, A, Em, G, D, A, Em, G, D, A, Em, G,...","[F#m, A, E, B, F#m, A, E, B, F#m, A, E, B, F#m...","[6, 1, 5, None, 6, 1, 5, None, 6, 1, 5, None, ..."
1,Verse 1,"[Em, G, D, A7sus4, Em, G, D, A7sus4]",27596,2,A,"[Em, G, D, A, Em, G, D, A]","[F#m, A, E, B, F#m, A, E, B]","[6, 1, 5, None, 6, 1, 5, None]"
2,Verse 2,"[Em, G, D, A7sus4, Em, G, D, A7sus4, Em, G, D,...",27596,2,A,"[Em, G, D, A, Em, G, D, A, Em, G, D, A, Em, G,...","[F#m, A, E, B, F#m, A, E, B, F#m, A, E, B, F#m...","[6, 1, 5, None, 6, 1, 5, None, 6, 1, 5, None, ..."
3,Chorus,"[C, Em, G, Em, C, Em, G, Em, C, Em, G, Em, C, ...",27596,2,A,"[C, Em, G, Em, C, Em, G, Em, C, Em, G, Em, C, ...","[D, F#m, A, F#m, D, F#m, A, F#m, D, F#m, A, F#...","[4, 6, 1, 6, 4, 6, 1, 6, 4, 6, 1, 6, 4, 6, 1, 6]"
4,Verse 3,"[Em, G, D, A7sus4, Em, G, D, A7sus4, Em, G, D,...",27596,2,A,"[Em, G, D, A, Em, G, D, A, Em, G, D, A, Em, G,...","[F#m, A, E, B, F#m, A, E, B, F#m, A, E, B, F#m...","[6, 1, 5, None, 6, 1, 5, None, 6, 1, 5, None, ..."


In [16]:
md[md.index == song['id'][0]]  # using id from song table to read from metadata df

,song_id,song_name,artist_id,artist_name,type,version,votes,rating,date,status,preset_id,tonality_name,verified,artist_url,tab_url,type_name,hits
id,,,,,,,,,,,,,,,,,
27596,19556,Wonderwall,734,Oasis,Chords,2,5205,4.77542,1018656000,approved,2332,F#m,0,https://www.ultimate-guitar.com/artist/oasis_734,https://tabs.ultimate-guitar.com/tab/oasis/won...,Chords,7935922


In [17]:
t = ['D#', 'G#', 'C', 'F']
get_key(t)

'A#'

In [18]:
key_chord_mapping

,A,A#,Bb,B,C,C#,Db,D,D#,Eb,E,F,F#,Gb,G,G#,Ab
Degree,,,,,,,,,,,,,,,,,
1,A,A#,Bb,B,C,C#,Db,D,D#,Eb,E,F,F#,Gb,G,G#,Ab
2,Bm,Cm,Cm,C#m,Dm,D#m,Ebm,Em,Fm,Fm,F#m,Gm,G#m,Abm,Am,A#m,Bbm
3,C#m,Dm,Dm,D#m,Em,Fm,Fm,F#m,Gm,Gm,G#m,Am,A#m,Bbm,Bm,Cm,Cm
4,D,D#,Eb,E,F,F#,Gb,G,G#,Ab,A,Bb,B,B,C,C#,Db
5,E,F,F,F#,G,G#,Ab,A,A#,Bb,B,C,C#,Db,D,D#,Eb
6,F#m,Gm,Gm,G#m,Am,A#m,Bbm,Bm,Cm,Cm,C#m,Dm,D#m,Ebm,Em,Fm,Fm
7,G#dim,Adim,Adim,A#dim,Bdim,Cdim,Cdim,C#dim,Ddim,Ddim,D#dim,Edim,Fdim,Fdim,F#dim,Gdim,Gdim
